# Tutorial PAMAP2 with mcfly

This turorial is intended to talk you through the functionalities of mcfly. As an example dataset we use the publicly available [PAMAP2 dataset](https://archive.ics.uci.edu/ml/datasets/PAMAP2+Physical+Activity+Monitoring). It contains time series data from movement sensors worn by nine individuals. The data is labelled with the activity types that these individuals did and the aim is to train and evaluate a *classifier*.

Before you can start, please make sure you install mcfly (see the [mcfly installation page](https://github.com/NLeSC/mcfly)).

## Import required Python modules

In [1]:
import sys
import os
import numpy as np
import pandas as pd
# mcfly
from mcfly import modelgen, find_architecture, storage
np.random.seed(2)

Using TensorFlow backend.


In [2]:
sys.path.insert(0, os.path.abspath('../..'))
from utils import tutorial_pamap2

## Download data pre-procesed data

We have created a function for you to fetch the preprocessed data. Please specify the `directory_to_extract_to` in the code below and then execute the cell. This will download the preprocessed data into the directory in the `data` subdirectory. The output of the function is the path where the preprocessed data was stored.

In [3]:
# Specify in which directory you want to store the data:
directory_to_extract_to = '.'

In [4]:
data_path = tutorial_pamap2.download_preprocessed_data(directory_to_extract_to)

Data already downloaded and extracted.


## Load the pre-processed data

Load the preprocessed data as stored in Numpy-files. Please note that the data has already been split up in a training (training), validation (val), and test subsets. It is common practice to call the input data X and the labels y.

In [5]:
X_train, y_train_binary, X_val, y_val_binary, X_test, y_test_binary, labels = tutorial_pamap2.load_data(data_path)

Data X and labels y are of type Numpy array. In the cell below we inspect the shape of the data. As you can see the shape of X is expressed as a Python tuple containing: the number of samples, length of the time series, and the number of channels for each sample. Similarly, the shape of y is represents the number of samples and the number of classes (unique labels). Note that y has the format of a binary array where only the correct class for each sample is assigned a 1. This is called one-hot-encoding.

In [6]:
print('x shape:', X_train.shape)
print('y shape:', y_train_binary.shape)

x shape: (15718, 512, 9)
y shape: (15718, 12)


The data is split between train test and validation.

In [7]:
print('train set size:', X_train.shape[0])
print('validation set size:', X_val.shape[0])
print('test set size:', X_test.shape[0])

train set size: 15718
validation set size: 100
test set size: 1000


Let's have a look at the distribution of the labels:

In [8]:
frequencies = y_train_binary.mean(axis=0)
frequencies_df = pd.DataFrame(frequencies, index=labels, columns=['frequency'])
frequencies_df

,frequency
lying,0.103257
sitting,0.098486
standing,0.102748
walking,0.129978
running,0.049688
cycling,0.087416
nordic_walking,0.101285
ascending_stairs,0.046507
descending_stairs,0.037982
vaccuum_cleaning,0.091806


### *Question 1: How many channels does this dataset have?*
### *Question 2: What is the least common activity label in this dataset?*

    

## Generate models

First step in the development of any deep learning model is to create a model architecture. As we do not know what architecture is best for our data we will create a set of random models to investigate which architecture is most suitable for our data and classification task. This process, creating random models, checking how good they are and then selecting the best one is called a 'random search'. A random search is considered to be the most robust approach to finding a good model. You will need to specificy how many models you want to create with argument 'number_of_models'. See for a full overview of the optional arguments the function documentation of modelgen.generate_models by running `modelgen.generate_models?`.

##### What number of models to select?
This number differs per dataset. More models will give better results but it will take longer to evaluate them. For the purpose of this tutorial we recommend trying only 2 models to begin with. If you have enough time you can try a larger number of models, e.g. 10 or 20 models. Because mcfly uses random search, you will get better results when using more models.

In [9]:
num_classes = y_train_binary.shape[1]

models = modelgen.generate_models(X_train.shape,
                                  number_of_classes=num_classes,
                                  number_of_models = 2)

# Inspect the models
We can have a look at the models that were generated. The layers are shown as table rows. Most common layer types are 'Convolution' and 'LSTM' and 'Dense'. For more information see the [mcfly user manual](https://github.com/NLeSC/mcfly/wiki/User-manual) and the [tutorial cheat sheet](https://github.com/NLeSC/mcfly-tutorial/blob/master/cheatsheet.md). The summary also shows the data shape of each layer output and the number of parameters that are trained within this layer.

In [10]:
models_to_print = range(len(models))
for i, item in enumerate(models):
    if i in models_to_print:
        model, params, model_types = item
        print("-------------------------------------------------------------------------------------------------------")
        print("Model " + str(i))
        print(" ")
        print("Hyperparameters:")
        print(params)
        print(" ")
        print("Model description:")
        model.summary()
        print(" ")
        print("Model type:")
        print(model_types)
        print(" ")

-------------------------------------------------------------------------------------------------------
Model 0
 
Hyperparameters:
{'learning_rate': 0.08360289270402858, 'filters': array([85, 17, 44]), 'regularization_rate': 0.0022439468517196116, 'fc_hidden_nodes': 443}
 
Model description:
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_1 (BatchNorma (None, 512, 9)        36          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)  (None, 512, 85)       2380        batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNorma (None, 512, 85)       340         convolution1d_1[0][0]            


### *Question 3: Can you guess what hyperparameter 'learning rate' stands for?*

## Compare models
Now that the model architectures have been generated it is time to compare the models by training them on a subset of the training data and evaluating the models on the validation subset. This will help us to choose the best candidate model. The performance results for the models are stored in a json file, which we will visually inspect later on.

In [11]:
# Define directory where the results, e.g. json file, will be stored

resultpath = os.path.join(directory_to_extract_to, 'data/models')
if not os.path.exists(resultpath):
        os.makedirs(resultpath)

We are now going to train each of the models that we generated. On the one hand we want to train them as quickly as possible in order to be able to pick the best one as soon as possible. On the other hand we have to train each model long enough to get a good impression of its potential.

We can influence the train time by adjusting the number of data samples that are used. This can be set with the argument 'subset_size'. We can also adjust the number of times the subset is iterated over. This number is called an epoch. We recommend to start with no more than 5 epochs and a maximum subset size of 300. You can experiment with these numbers.

In [12]:
outputfile = os.path.join(resultpath, 'modelcomparison.json')
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                           X_val, y_val_binary,
                                                                           models,nr_epochs=5,
                                                                           subset_size=300,
                                                                           verbose=True,
                                                                           outputfile=outputfile)
print('Details of the training process were stored in ',outputfile)

Training model 0 CNN
Train on 300 samples, validate on 100 samples
Epoch 1/5
300/300 [==============================] - 8s - loss: 493.5578 - acc: 0.3167 - val_loss: 464.6164 - val_acc: 0.1600
Epoch 2/5
300/300 [==============================] - 7s - loss: 235.2949 - acc: 0.4667 - val_loss: 96.7883 - val_acc: 0.2000
Epoch 3/5
300/300 [==============================] - 6s - loss: 48.3533 - acc: 0.5600 - val_loss: 30.3034 - val_acc: 0.3100
Epoch 4/5
300/300 [==============================] - 8s - loss: 14.5919 - acc: 0.6167 - val_loss: 20.5024 - val_acc: 0.2000
Epoch 5/5
300/300 [==============================] - 9s - loss: 7.6085 - acc: 0.5867 - val_loss: 16.3494 - val_acc: 0.1600
Training model 1 CNN
Train on 300 samples, validate on 100 samples
Epoch 1/5
300/300 [==============================] - 37s - loss: 2.6673 - acc: 0.3467 - val_loss: 3.4942 - val_acc: 0.0900
Epoch 2/5
300/300 [==============================] - 33s - loss: 2.4847 - acc: 0.6467 - val_loss: 3.6901 - val_acc: 0.190

### *Question 4: What do the terms 'acc' and 'loss' in the output refer to?*

# Inspect model performance (Visualization)

Details about the learning process can be visualized. To use mcfly's visualization, take the following steps:
- Clone the mcfly github repository (if you haven't done so already for visualization)

 `git clone https://github.com/NLeSC/mcfly`


- navigate to the html folder:

 `cd mcfly/html`


- Start a web server. This can be done in various ways, for example:
 - for python 3 you can use: `python3 -m http.server`
 - for python 2 you can use: `python2 -m SimpleHTTPServer`

Notice the port number the web server is serving on. This is usually 8000.
With a web browser, navigate to [localhost:8000](localhost:8000). There you can upload the json file that contains the details of the training process. The following line of code shows the path to this file:

In [13]:
outputfile

'./data/models/modelcomparison.json'

### *Question 5:  Look at the visualization. Which model performs best?*
### *Question 6:  Did you train all models with a sufficient number of iterations?*

# Inspect model performance (table)

Let's compare the performance of the models by showing the results as a table.

In [14]:
modelcomparisons = pd.DataFrame({'model':[str(params) for model, params, model_types in models],
                       'train_acc': [history.history['acc'][-1] for history in histories],
                       'train_loss': [history.history['loss'][-1] for history in histories],
                       'val_acc': [history.history['val_acc'][-1] for history in histories],
                       'val_loss': [history.history['val_loss'][-1] for history in histories]
                       })
modelcomparisons.to_csv(os.path.join(resultpath, 'modelcomparisons.csv'))

modelcomparisons

,model,train_acc,train_loss,val_acc,val_loss
0,"{'learning_rate': 0.08360289270402858, 'filter...",0.586667,7.608480,0.16,16.349435
1,"{'learning_rate': 0.0012641156505383837, 'filt...",0.856667,1.916203,0.19,3.643516


# Choose the best model
Now that we found an effective architecture, we can choose the most promising model. For example, we can choose the model with the highest accuracy on the validation data set. To maximize this models performance, we will train this model on more data and more epochs.

In [15]:
best_model_index = np.argmax(val_accuracies)
best_model, best_params, best_model_types = models[best_model_index]
print('Model type and parameters of the best model:')
print(best_model_types)
print(best_params)

Model type and parameters of the best model:
CNN
{'learning_rate': 0.0012641156505383837, 'filters': array([77, 14, 52, 61, 48, 43, 68, 77]), 'regularization_rate': 0.00035101028011951283, 'fc_hidden_nodes': 1359}


## Train the best model on the full dataset

Now that we have identified the best model architecture out of our random pool of models we can continue by training the model on the full training set.

In [16]:
#We make a copy of the model, to start training from fresh
nr_epochs = 1
datasize = X_train.shape[0] #We're going to train the model on the complete data set
history = best_model.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_val, y_val_binary))

Train on 15718 samples, validate on 100 samples
Epoch 1/1
15718/15718 [==============================] - 1734s - loss: 0.9841 - acc: 0.8906 - val_loss: 0.7748 - val_acc: 0.8700


### *Question 7: Why does it take longer to train the best model for one epoch now than when we were  comparing model archicatures earlier on?*

### *Question 8: Do you think it is useful to train with more than 1 epoch?*

### Saving, loading and comparing reloaded model with orignal model

The model can be saved for future use. The savemodel function will save two separate files: a json file for the architecture and a npy (numpy array) file for the weights.

In [18]:
modelname = 'my_bestmodel'

In [19]:
storage.savemodel(best_model,resultpath,modelname)

('./data/modelsmy_bestmodel_architecture.json',
 './data/modelsmy_bestmodel_weights')

In [20]:
model_reloaded = storage.loadmodel(resultpath,modelname)

The model has been reloaded. Let's investigate whether it gives the same probability estimates as the original model in a small subset of the validation data.

In [21]:
datasize = 10
probs_original = best_model.predict_proba(X_val[:datasize,:,:],batch_size=1)
probs_reloaded = model_reloaded.predict_proba(X_val[:datasize,:,:],batch_size=1)

10/10 [==============================] - 0s     


In [22]:
(probs_reloaded == probs_original).all()

True

## Investigate model predictions

Note that the objects `models`, `best_model_fullytrained` and `best_model` that resulted from the mcfly functions are Keras objects. This means that you can use Keras functions on the objects, for example  `.predict`, (which when given the data, outputs the predictions for each sample) and `.evaluate` (which when given the data and the labels computes how well this model performs) . These functions are all documented in the [Keras documentation](https://keras.io/). 

In [23]:
## Inspect model predictions on validation data
datasize = X_val.shape[0]
probs = best_model.predict_proba(X_val[:datasize,:,:],batch_size=1)

100/100 [==============================] - 2s     


Let's have a look at the confusion matrix

In [27]:
#columns are predicted, rows are truth
predicted = probs.argmax(axis=1)
y_index = y_val_binary.argmax(axis=1)
confusion_matrix = pd.crosstab(pd.Series(y_index), pd.Series(predicted))
confusion_matrix.index = [labels[i] for i in confusion_matrix.index]
confusion_matrix.columns = [labels[i] for i in confusion_matrix.columns]
confusion_matrix.reindex(columns=["bla" for l in labels], fill_value=0)
confusion_matrix

,lying,sitting,standing,walking,running,cycling,nordic_walking,ascending_stairs,descending_stairs,vaccuum_cleaning,ironing,rope_jumping
lying,13,0,0,0,0,0,0,0,0,0,0,2
sitting,0,9,0,0,0,0,0,0,0,0,0,0
standing,0,1,9,0,0,0,0,0,0,0,0,0
walking,0,0,0,15,0,0,0,1,0,0,0,0
running,0,0,0,0,2,0,0,0,0,0,0,1
cycling,0,0,0,0,0,6,0,1,0,0,0,0
nordic_walking,0,0,0,0,0,0,8,0,0,0,0,0
ascending_stairs,0,0,0,0,0,0,0,2,0,0,0,0
descending_stairs,0,0,0,0,0,0,0,0,1,0,0,0
vaccuum_cleaning,0,0,0,0,0,1,1,0,1,6,0,0


In [25]:
## Test on Testset
score_test = best_model.evaluate(X_test, y_test_binary, verbose=True)
print('Score of best model: ' + str(score_test))

1000/1000 [==============================] - 11s    
Score of best model: [0.73806815814971927, 0.873]
